# WAN ClownShark Realistic Waifu Setup

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/USERNAME/REPO/blob/BRANCH/WAN_ClownShark_Realistic_Waifu_Setup.ipynb)

Stručný popis kroků: Drive → Install → Models → Workflow → Launch (Cloudflare).


In [ ]:

import os
import sys
import json
import time
import subprocess
import shutil
import glob
from pathlib import Path
from google.colab import drive, files, userdata, output

INSTALL_DIR = "/content"
COMFY_DIR = f"{INSTALL_DIR}/ComfyUI"
DRIVE_MOUNT = "/content/drive"
DRIVE_BASE = f"{DRIVE_MOUNT}/MyDrive/ComfyAssets"
DIRS = {
    "checkpoints": f"{DRIVE_BASE}/checkpoints",
    "clip": f"{DRIVE_BASE}/clip",
    "vae": f"{DRIVE_BASE}/vae",
    "loras": f"{DRIVE_BASE}/loras",
    "upscale": f"{DRIVE_BASE}/upscale_models",
    "lama": f"{DRIVE_BASE}/lama",
    "workflows": f"{COMFY_DIR}/user/workflows",
}

CUSTOM_REPOS = [
    {"name": "ComfyUI-Impact-Pack", "url": "https://github.com/ltdrdata/ComfyUI-Impact-Pack.git", "path": f"{COMFY_DIR}/custom_nodes/ComfyUI-Impact-Pack"},
    {"name": "rgthree-comfy", "url": "https://github.com/rgthree/rgthree-comfy.git", "path": f"{COMFY_DIR}/custom_nodes/rgthree-comfy"},
    {"name": "ComfyUI-Clownshark", "url": "https://github.com/RES4LYF/ComfyUI-Clownshark.git", "path": f"{COMFY_DIR}/custom_nodes/ComfyUI-Clownshark"},
    {"name": "ComfyUI-Easy-Use", "url": "https://github.com/yolain/ComfyUI-Easy-Use.git", "path": f"{COMFY_DIR}/custom_nodes/ComfyUI-Easy-Use"},
    {"name": "ComfyUI_essentials", "url": "https://github.com/cubiq/ComfyUI_essentials.git", "path": f"{COMFY_DIR}/custom_nodes/ComfyUI_essentials"},
    {"name": "ComfyUI-LayerStyle", "url": "https://github.com/Extraltodeus/ComfyUI-LayerStyle.git", "path": f"{COMFY_DIR}/custom_nodes/ComfyUI-LayerStyle"},
    {"name": "ComfyUI-Use-Everywhere", "url": "https://github.com/cg-use/ComfyUI-Use-Everywhere.git", "path": f"{COMFY_DIR}/custom_nodes/ComfyUI-Use-Everywhere"},
    {"name": "ComfyUI-Art-Venture", "url": "https://github.com/yolain/ComfyUI-Art-Venture.git", "path": f"{COMFY_DIR}/custom_nodes/ComfyUI-Art-Venture"},
    {"name": "comfyui-vrgamedevgirl", "url": "https://github.com/vrgamedevgirl/comfyui-vrgamedevgirl.git", "path": f"{COMFY_DIR}/custom_nodes/comfyui-vrgamedevgirl"},
]

REQUIRED_FILES = [
    ("checkpoints", "Wan2.2/Wan2_2-T2V-A14B-LOW_fp8_e4m3fn_scaled_KJ.safetensors"),
    ("clip", "umt5_xxl_fp8_e4m3fn_scaled.safetensors"),
    ("vae", "wan_2.1_vae.safetensors"),
    ("loras", "Wan2.2-Lightning_T2V-v1.1-A14B-4steps-lora_LOW_fp16.safetensors"),
    ("loras", "spade_wan_v2_t2v_14b_000002750_low_noise.safetensors"),
    ("loras", "Instagirlv2.5/Instagirlv2.5-LOW.safetensors"),
    ("upscale", "4xNomosWebPhoto_esrgan.pth"),
    ("lama", "big-lama.pt"),
]

def run(cmd: list[str]) -> None:
    print(f"$ {' '.join(cmd)}")
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.stdout:
        print(result.stdout)
    if result.stderr:
        print(result.stderr, file=sys.stderr)
    if result.returncode != 0:
        raise RuntimeError(f"Command failed with exit code {result.returncode}: {' '.join(cmd)}")

for path in DIRS.values():
    os.makedirs(path, exist_ok=True)


In [ ]:

print("Mounting Google Drive…")
drive.mount(DRIVE_MOUNT, force_remount=False)
os.makedirs(DRIVE_BASE, exist_ok=True)
test_file = Path(DRIVE_BASE) / "._colab_write_test"
try:
    with open(test_file, "w") as f:
        f.write("ok")
    test_file.unlink()
    print(f"Drive mounted at {DRIVE_MOUNT} and write test passed.")
except Exception as exc:
    print(f"Drive write test failed: {exc}")


In [ ]:

if not Path(COMFY_DIR).exists():
    run(["git", "clone", "https://github.com/comfyanonymous/ComfyUI.git", COMFY_DIR])
else:
    run(["git", "-C", COMFY_DIR, "pull", "--rebase", "--autostash"])

for repo in CUSTOM_REPOS:
    target = Path(repo["path"])
    target.parent.mkdir(parents=True, exist_ok=True)
    if not target.exists():
        run(["git", "clone", repo["url"], str(target)])
    else:
        run(["git", "-C", str(target), "pull", "--rebase", "--autostash"])

req_file = Path(COMFY_DIR) / "requirements.txt"
if req_file.exists():
    run([sys.executable, "-m", "pip", "install", "-q", "-r", str(req_file)])

custom_reqs = sorted(Path(COMFY_DIR, "custom_nodes").glob("*/requirements.txt"))
for req in custom_reqs:
    run([sys.executable, "-m", "pip", "install", "-q", "-r", str(req)])

run([sys.executable, "-m", "pip", "install", "-q", "huggingface_hub", "requests", "safetensors", "accelerate", "einops", "timm", "opencv-python", "pillow", "cloudflared"])


In [ ]:

models_root = Path(COMFY_DIR) / "models"
models_root.mkdir(parents=True, exist_ok=True)
model_links = {
    "checkpoints": DIRS["checkpoints"],
    "clip": DIRS["clip"],
    "vae": DIRS["vae"],
    "loras": DIRS["loras"],
    "upscale_models": DIRS["upscale"],
    "lama": DIRS["lama"],
}

for name, drive_path in model_links.items():
    target = models_root / name
    drive_target = Path(drive_path)
    drive_target.mkdir(parents=True, exist_ok=True)
    if target.exists() or target.is_symlink():
        if target.is_symlink():
            current = os.readlink(target)
            if current != str(drive_target):
                target.unlink()
        elif target.is_dir():
            try:
                if not os.path.samefile(target, drive_target):
                    shutil.rmtree(target)
            except FileNotFoundError:
                shutil.rmtree(target)
        else:
            target.unlink()
    if not target.exists():
        os.symlink(drive_target, target)

Path(DIRS["checkpoints"]).joinpath("Wan2.2").mkdir(parents=True, exist_ok=True)
Path(DIRS["loras"]).joinpath("Instagirlv2.5").mkdir(parents=True, exist_ok=True)


In [ ]:

from typing import Iterable, Optional, Tuple

try:
    from huggingface_hub import hf_hub_download
except Exception:
    hf_hub_download = None

missing: list[Tuple[str, str]] = []
token = userdata.get('HF_TOKEN') or os.environ.get('HF_TOKEN')

# Placeholder for potential Hugging Face sources. Populate when URLs are known.
HF_CANDIDATES: dict[Tuple[str, str], list[Tuple[str, str]]] = {}

def ensure_file(category: str, relpath: str, candidates: Optional[Iterable[Tuple[str, str]]] = None) -> None:
    dst_dir = Path(DIRS[category])
    dst = dst_dir / relpath
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists():
        print(f"Found {dst}")
        return
    attempted = False
    if token and hf_hub_download and candidates:
        for repo_id, filename in candidates:
            attempted = True
            try:
                downloaded = hf_hub_download(repo_id=repo_id, filename=filename, token=token, local_dir=str(dst.parent))
                final_path = Path(downloaded)
                if final_path != dst:
                    final_path.rename(dst)
                print(f"Downloaded {dst}")
                return
            except Exception as exc:
                print(f"Download failed for {repo_id}/{filename}: {exc}")
    if attempted:
        print(f"Unable to download {relpath}")
    missing.append((category, relpath))

for category, relpath in REQUIRED_FILES:
    candidates = HF_CANDIDATES.get((category, relpath))
    # TODO: add Hugging Face (repo_id, filename) pairs in HF_CANDIDATES when available.
    ensure_file(category, relpath, candidates)

if missing:
    print("The following files are missing and require upload:")
    for category, relpath in missing:
        print(f" - {Path(relpath).name}: {DIRS[category]}/{relpath}")
    print("Opening upload dialog…")
    uploaded = files.upload()
    for filename, data in uploaded.items():
        basename = Path(filename).name
        for entry in list(missing):
            category, relpath = entry
            if Path(relpath).name == basename:
                dst = Path(DIRS[category]) / relpath
                dst.parent.mkdir(parents=True, exist_ok=True)
                with open(dst, "wb") as f:
                    f.write(data)
                print(f"Saved upload to {dst}")
                missing.remove(entry)
                break
    if missing:
        print("Still missing after upload:")
        for category, relpath in missing:
            print(f" - {DIRS[category]}/{relpath}")
else:
    print("All required assets are present.")


In [ ]:

WORKFLOW_JSON = r""""""
workflow_path = Path(DIRS["workflows"]) / "ClownShark Realistic Waifu Generator v2.json"
workflow_path.parent.mkdir(parents=True, exist_ok=True)
if workflow_path.exists():
    backup_path = workflow_path.with_suffix(workflow_path.suffix + ".bak")
    shutil.copy2(workflow_path, backup_path)
    print(f"Existing workflow backed up to {backup_path}")
with open(workflow_path, "w", encoding="utf-8") as f:
    f.write(WORKFLOW_JSON)
size = workflow_path.stat().st_size
print(f"Workflow saved to {workflow_path} ({size} bytes)")


In [ ]:

for proc in ("python", "cloudflared"):
    try:
        run(["pkill", "-f", proc])
    except Exception:
        pass

proc_ui = subprocess.Popen([
    sys.executable,
    "main.py",
    "--listen",
    "0.0.0.0",
    "--port",
    "8188",
    "--enable-cors-header",
], cwd=COMFY_DIR, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

print("ComfyUI starting on http://127.0.0.1:8188")

time.sleep(5)

proc_cf = subprocess.Popen([
    "cloudflared",
    "tunnel",
    "--url",
    "http://localhost:8188",
    "--no-autoupdate",
], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

public_url = None
start_time = time.time()
while True:
    if proc_cf.stdout is None:
        break
    line = proc_cf.stdout.readline()
    if not line:
        if time.time() - start_time > 60:
            print("Timeout waiting for Cloudflare URL. Try rerunning the cell.")
            break
        time.sleep(1)
        continue
    print(line.strip())
    if "trycloudflare.com" in line:
        for token in line.split():
            if token.startswith("https://") and "trycloudflare.com" in token:
                public_url = token.strip()
                break
    if public_url:
        print(f"Public URL: {public_url}")
        break

print("Local URL: http://127.0.0.1:8188")


In [ ]:

models_root = Path(COMFY_DIR) / "models"
print("ComfyUI models directory tree:")
for root, dirs, files in os.walk(models_root):
    level = Path(root).relative_to(models_root).parts
    indent = "  " * len(level)
    print(f"{indent}{Path(root).name}/")
    for file in sorted(files):
        print(f"{indent}  {file}")

print("
Validated assets:")
for category, relpath in REQUIRED_FILES:
    path = Path(DIRS[category]) / relpath
    print(f"{'[OK]' if path.exists() else '[MISSING]'} {path}")

workflow_path = Path(DIRS["workflows"]) / "ClownShark Realistic Waifu Generator v2.json"
if workflow_path.exists():
    with open(workflow_path, "r", encoding="utf-8") as f:
        snippet = f.read(200)
    print(f"
Workflow file located: {workflow_path}")
    print(f"Preview: {snippet[:120]}...")
else:
    print("Workflow file not found!")


## Rychlý návod k použití

1. Spusť buňky 2–8.
2. Otevři vypsané `Public URL`.
3. V ComfyUI → `Load` → najdi `ClownShark Realistic Waifu Generator v2.json` v menu Workflows (je už uložen).

Poznámky:
- LoRA zap/vyp v `Power Lora Loader`.
- Maskování přes `Mask here!` PreviewBridge.
- Exporty `Final Export of Lust` a `Final Export of Lust_Corrected`.
